In [1]:
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=FutureWarning)
    import tensorflow as tf
import numpy as np
import IPython
import random
import glob
import cv2
import matplotlib.pyplot as plt
import threading
import time
# images plotting function
def plotImg(imgs, titles = None):
    fig = plt.figure()
    for i in range(len(imgs)):
        ax = fig.add_subplot(1, len(imgs), i + 1)
        if titles != None:
            ax.title.set_text(titles[i])
        plt.imshow(imgs[i])
        plt.axis('off')
    plt.show()
# Set parameters
frame = None
threadRunFlg = True
nums = ['0','0','0','0','0','0']
fpsCnt = 0
lastFpsNum = 0
fpsVal = None
predClass = None
pause = False
chpk_dir = 'checkpoints/cnnGesture/'
chpk_name = 'model'
# font stype
stype = cv2.FONT_HERSHEY_SIMPLEX
# red:(0,0,255) blue:(255,0,0) pink:(255,0,255) black:(0,0,0) white:(255,255,255) green: (0,255,0)
title_color = (255,0,255)
content_color = (0,0,255)
sclale = .5
thickness = 1
# restore checkpoint
sessRes = tf.Session()
saver = tf.train.import_meta_graph(chpk_dir + 'model.meta')
saver.restore(sessRes, tf.train.latest_checkpoint(chpk_dir))
# restore tensors
img_holder = sessRes.graph.get_tensor_by_name('img_holder:0')
lbl_holder =sessRes.graph.get_tensor_by_name('lbl_holder:0')
train = sessRes.graph.get_tensor_by_name('train_bool:0')
predict = sessRes.graph.get_tensor_by_name('prediction:0')
probabilities = sessRes.graph.get_tensor_by_name('probabilities:0')
# do prediction using threading way
def predicionjob():
    global predClass, threadRunFlg, nums, fpsCnt, pause, sessRes
    if threadRunFlg:
        if fpsCnt and not pause:
            pred, prob = sessRes.run([predict, probabilities], feed_dict = {img_holder: [cv2.cvtColor(frame[90:390, 135:505], cv2.COLOR_BGR2RGB)], train: False})
            predClass = pred[0]
            for i in range(6):
                nums[i] = '%.2f' % prob[0][i]
        else:
            predClass = None
            for i in range(6):
                nums[i] = '0'
        threading.Timer(0.1, predicionjob).start()
# fps calculation
def calculateFPS():
    global fpsVal, lastFpsNum
    if threadRunFlg:
        if lastFpsNum == 0:
            lastFpsNum = fpsCnt
        else: 
            fpsVal = (fpsCnt - lastFpsNum) / 0.5
            lastFpsNum = fpsCnt
        threading.Timer(0.5, calculateFPS).start()
# camera prepration
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
predicionjob()
calculateFPS()
# Start time
start = time.time()
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    # Display the resulting frame
    cv2.rectangle(frame,(132,85),(508,392),(0,255,0),3)
    cv2.putText(frame,'Real Time Gesture Recognition', (10,15), stype, sclale, title_color, thickness)
    cv2.putText(frame, 'FPS:', (565,15), stype, sclale, content_color, thickness)
    cv2.putText(frame, str(fpsVal), (600,15), stype, sclale, content_color, thickness)
    cv2.putText(frame,'Prediction: ', (10,30), stype, sclale, title_color, thickness)
    cv2.putText(frame, str(predClass), (100,30), stype, sclale, content_color, thickness)
    cv2.putText(frame,'Prob G0:', (10,45), stype, sclale, title_color, thickness)
    cv2.putText(frame, str(nums[0]), (85,45), stype, sclale, content_color, thickness)
    cv2.putText(frame,'Prob G1:', (10,60), stype, sclale, title_color, thickness)
    cv2.putText(frame, str(nums[1]), (85,60), stype, sclale, content_color, thickness)
    cv2.putText(frame,'Prob G2:', (10,75), stype, sclale, title_color, thickness)
    cv2.putText(frame, str(nums[2]), (85,75), stype, sclale, content_color, thickness)
    cv2.putText(frame,'Prob G3:', (10,90), stype, sclale, title_color, thickness)
    cv2.putText(frame, str(nums[3]), (85,90), stype, sclale, content_color, thickness)
    cv2.putText(frame,'Prob G4:', (10,105), stype, sclale, title_color, thickness)
    cv2.putText(frame, str(nums[4]), (85,105), stype, sclale, content_color, thickness)
    cv2.putText(frame,'Prob G5:', (10,120), stype, sclale, title_color, thickness)
    cv2.putText(frame, str(nums[5]), (85,120), stype, sclale, content_color, thickness)
    # show
    cv2.imshow('frame',frame)
    k = cv2.waitKey(1)
    if k & 0xFF == ord('q'):
        threadRunFlg = False
        break
    elif k & 0xFF == ord('p'):
        pause = not pause
    fpsCnt += 1
end = time.time()
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

INFO:tensorflow:Restoring parameters from checkpoints/cnnGesture/model


{'restart': True, 'status': 'ok'}